In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Extract

The first file we are going to be using is a list of breweries in the US. This csv has columns for name, address, website, establishment type, state, and number of breweries in that state.
The second file we are going to be using is also a csv of state population data from the US census. This file also contains columns giving the latitude and longitude of the given state's capital. Both of these datasets were available on Kaggle.com.

In [2]:
pop_file = "Resources/2019_Census_US_Population_Data_By_State_Lat_Long.csv"
pop_df = pd.read_csv(pop_file)
pop_df.head()

,STATE,POPESTIMATE2019,lat,long
0,Alabama,4903185,32.377716,-86.300568
1,Alaska,731545,58.301598,-134.420212
2,Arizona,7278717,33.448143,-112.096962
3,Arkansas,3017804,34.746613,-92.288986
4,California,39512223,38.576668,-121.493629


In [3]:
print(f'There are {pop_df["STATE"].count()} rows in the population dataset.')

There are 51 rows in the population dataset.


In [4]:
brew_file = "Resources/breweries_us 2.csv"
brew_df = pd.read_csv(brew_file)
brew_df.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [5]:
print(f'There are {brew_df["state"].nunique()} unique states in the brewery dataset.')

There are 51 unique states in the brewery dataset.


# Transform

For the population data set, we are going to copy only the state and population columns, essentially dropping the latitude and longitude columns. Next we are going to drop all rows related to the District of Columbia, since the breweries dataset does not have any breweries in the District of Columbia. We are also going to rename the columns to be consistent with the table we are going to make in SQL.

In [6]:
# Create a filtered dataframe from specific columns
pop_cols = ["STATE", "POPESTIMATE2019"]
pop_transformed= pop_df[pop_cols].copy()
pop_transformed.head()

,STATE,POPESTIMATE2019
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [7]:
# Rename the column headers
pop_transformed = pop_transformed.rename(columns={"STATE": "state",
                                                          "POPESTIMATE2019": "population"
                                                          })
pop_transformed.head(20)

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
8,District of Columbia,705749
9,Florida,21477737


In [8]:
#drop District of Columbia
pop_transformed = pop_transformed[pop_transformed.state != "District of Columbia"]
pop_transformed.head(20)

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
9,Florida,21477737
10,Georgia,10617423


In [9]:
#Reset Indices
pop_transformed=pop_transformed.reset_index
pop_transformed.head(20)

AttributeError: 'function' object has no attribute 'head'